On the keras website, there is an example of time-series anomaly detection.  
Re-create this example in a notebook of your own, explaining the concepts.  
Clearly explain each keras function used, referring to the documentation.  
Include an introduction to your notebook, setting the context and
describing what the reader can expect as they read down through
the notebook.  
Include a conclusion section where you suggest improvements you
could make to the analysis in the notebook.  

Keras Team. Keras: the python deep learning api, 2022a. URL
https://keras.io/  
Keras Team. Timeseries anomaly detection using an autoencoder,
2022b. URL https://keras.io/examples/timeseries/timeseries_anomaly_detection/  
We will cover the main concepts
within this notebook in class, to give
you a starting point.

Keras is an open-source software library with a Python interface for artificial neural networks.  
Keras acts as an interface for the TensorFlow library, developed by the Google Brain team for the purposes of machine learning and artificial intelligence.  
Keras was created by Francois Chollet, a software engineer at Google, and the coding is regularly maintained on GitHub.  
Keras was created in order to better understand neural networks, which are a series of algorithms that intend to recognise trends in data which mimic how the human brain would perceive such trends from data.  
Keras allows users to develop neural networks on the Internet, on iOS and Android systems as well as the Java Virtual Machine.

In [ ]:
# importing modules to work with numbers, data frames, Keras and plots
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

Anomaly detection means developing a program that can detect any rarely occurring data points in the data set.  
We will provide Keras with data to read from via the Numenta Anomaly Benchmark (NAB) dataset.  
The NAB intends to provide artificial timeseries data which contains labelled anomalous periods of behaviour, all in order, all timestamped and all possessing single values.  
We will also use csv files:  
First, training is carried out via the small noise csv.  
Second, testing is carried out via the jumps up csv.  


In [ ]:
nab_url = "https://raw.githubusercontent.com/numenta/NAB/master/data/"

# training via small noise
df_small_noise_suffix = "artificialNoAnomaly/art_daily_small_noise.csv"
df_small_noise_url = nab_url + df_small_noise_suffix
df_small_noise = pd.read_csv(
    df_small_noise_url, parse_dates = True, index_col = "timestamp"
)

# testing via jumps up
df_daily_jumpsup_suffix = "artificialWithAnomaly/art_daily_jumpsup.csv"
df_daily_jumpsup_url = nab_url + df_daily_jumpsup_suffix
df_daily_jumpsup = pd.read_csv(
    df_daily_jumpsup_url, parse_dates = True, index_col = "timestamp"
)

# printing the data
print(df_small_noise.head())
print(df_daily_jumpsup.head())

References:
https://keras.io/examples/timeseries/timeseries_anomaly_detection/
https://huggingface.co/keras-io/time-series-anomaly-detection-autoencoder
https://curiousily.com/posts/anomaly-detection-in-time-series-with-lstms-using-keras-in-python/  
https://valueml.com/anomaly-detection-in-time-series-data-using-keras/  
https://towardsdatascience.com/time-series-of-price-anomaly-detection-with-lstm-11a12ba4f6d9  
https://en.wikipedia.org/wiki/Keras  
https://www.simplilearn.com/tutorials/deep-learning-tutorial/what-is-keras  
https://www.investopedia.com/terms/n/neuralnetwork.asp  
